In [13]:
!pip install -qU --progress-bar off --no-warn-conflicts Pillow langchain langchain-docling pymupdf langchain-community langchain_google_genai langchain_openai

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chat_models import ChatOpenAI

In [ ]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN'] = ""
os.environ['HF_TOKEN'] = ""
os.environ['OPENAI_API_KEY'] =""
source = "/content/drive/MyDrive/Colab Notebooks/2025 KB 부동산 보고서.pdf"

In [56]:
import os
import shutil

def empty_directory(directory_path):
    """
    지정된 폴더의 모든 내용을 삭제합니다.
    폴더가 존재하지 않는 경우 폴더를 새로 생성합니다.
    """
    # 폴더가 존재하는지 확인
    if os.path.exists(directory_path):
        # 폴더 내 모든 파일 및 하위 폴더 삭제
        for filename in os.listdir(directory_path):
            file_path = os.path.join(directory_path, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
                print(f"삭제됨: {file_path}")
            except Exception as e:
                print(f"오류 발생: {file_path} 삭제 중 - {e}")
        print(f"'{directory_path}' 폴더를 비웠습니다.")
    else:
        # 폴더가 없는 경우 새로 생성
        os.makedirs(directory_path)
        print(f"'{directory_path}' 폴더가 생성되었습니다.")

# 사용 예시
# 비울 폴더 경로 지정
folder_to_empty = "output"
empty_directory(folder_to_empty)

삭제됨: output/2025 KB 부동산 보고서_artifacts
삭제됨: output/images
삭제됨: output/2025 KB 부동산 보고서.md
'output' 폴더를 비웠습니다.


In [58]:
import os
import re
import base64
import io
import time
import logging
import numpy as np
from pathlib import Path
from typing import Dict, List, Optional, Tuple, Any
from PIL import Image

# Langchain 관련 임포트
from langchain_core.documents import Document
from langchain_core.messages import HumanMessage
from langchain.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_docling.loader import DoclingLoader, ExportType

# Docling 관련 임포트
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.document import DoclingDocument
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.datamodel.base_models import InputFormat
from docling_core.types.doc import ImageRefMode, PictureItem, TableItem

# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)


In [66]:
class ImageFilter:
    """
    문서 이미지 필터링을 위한 유틸리티 클래스

    이 클래스는 문서에서 추출된 이미지 중 분석할 가치가 있는 의미 있는 이미지(차트, 표, 다이어그램 등)를
    식별하고, 의미 없는 이미지(아이콘, 로고, 장식 요소 등)를 필터링하는 다양한 방법을 제공합니다.
    """

    @staticmethod
    def filter_by_size(img, min_width=150, min_height=150, min_area=30000):
        """
        이미지 크기 기반 필터링

        Args:
            img: PIL 이미지 객체
            min_width: 최소 너비 (픽셀)
            min_height: 최소 높이 (픽셀)
            min_area: 최소 면적 (픽셀 제곱)

        Returns:
            bool: 크기가 충분히 크면 True
        """
        width, height = img.size
        area = width * height

        return width >= min_width and height >= min_height and area >= min_area

    @staticmethod
    def filter_by_content(img, threshold=0.05):
        """
        이미지 콘텐츠 복잡성 기반 필터링

        복잡한 이미지(차트, 표 등)는 일반적으로 픽셀 변화가 크고 다양한 패턴을 가짐

        Args:
            img: PIL 이미지 객체
            threshold: 복잡성 임계값

        Returns:
            bool: 복잡성이 임계값보다 높으면 True
        """
        try:
            # 그레이스케일로 변환
            gray_img = img.convert('L')

            # 이미지 배열로 변환
            img_array = np.array(gray_img)

            # 픽셀 변화량 계산
            h_diff = np.abs(np.diff(img_array, axis=1))
            v_diff = np.abs(np.diff(img_array, axis=0))

            # 전체 변화량의 평균
            avg_diff = (np.mean(h_diff) + np.mean(v_diff)) / 2

            # 픽셀 값의 표준 편차 (복잡성 측정)
            std_dev = np.std(img_array)

            # 복잡성 점수
            complexity_score = (avg_diff * std_dev) / 255

            return complexity_score > threshold
        except Exception as e:
            logger.error(f"콘텐츠 필터링 오류: {e}")
            return True  # 오류 발생 시 기본적으로 포함

    @staticmethod
    def filter_by_text(img):
        """
        이미지에 텍스트가 충분히 있는지 확인
        단순한 아이콘은 텍스트가 적거나 없음

        Args:
            img: PIL 이미지 객체

        Returns:
            bool: 텍스트가 충분하면 True
        """
        # 텍스트가 있는 이미지는 일반적으로 흑백 대비가 큼
        gray_img = img.convert('L')
        img_array = np.array(gray_img)

        # 이미지 히스토그램의 분산이 클수록 텍스트가 있을 가능성이 높음
        histogram = np.histogram(img_array, bins=50)[0]
        hist_variance = np.var(histogram)

        return hist_variance > 1000  # 임의의 임계값

    @staticmethod
    def is_likely_icon(img, max_icon_area=10000, max_icon_dim=100):
        """
        이미지가 아이콘일 가능성이 높은지 확인

        Args:
            img: PIL 이미지 객체
            max_icon_area: 아이콘의 최대 면적
            max_icon_dim: 아이콘의 최대 차원 (너비 또는 높이)

        Returns:
            bool: 아이콘일 가능성이 높으면 True
        """
        width, height = img.size
        area = width * height

        # 작은 크기 확인
        if area > max_icon_area or width > max_icon_dim or height > max_icon_dim:
            return False

        # 정사각형에 가까운지 확인 (많은 아이콘이 정사각형)
        aspect_ratio = max(width, height) / min(width, height)
        is_square_like = aspect_ratio < 1.5

        # 단순한 색상 구성 확인
        colors = img.convert('RGB').getcolors(maxcolors=1024)
        few_colors = colors is not None and len(colors) < 32

        return is_square_like and few_colors

    @staticmethod
    def filter_by_context(markdown_content, image_position, window_size=300):
        """
        마크다운에서 이미지 참조 주변 텍스트 기반 필터링

        Args:
            markdown_content: 마크다운 텍스트
            image_position: 이미지 참조 위치
            window_size: 분석할 텍스트 창의 크기

        Returns:
            bool: 중요한 참조로 판단되면 True
        """
        # 이미지 주변 텍스트 추출
        start = max(0, image_position - window_size)
        end = min(len(markdown_content), image_position + window_size)
        context = markdown_content[start:end].lower()

        # 중요성 키워드 검색
        important_keywords = [
            '그림', '차트', '표', '다이어그램', 'figure', 'chart', 'table', 'diagram',
            'graph', '분석', 'analysis', '결과', 'result', '데이터', 'data'
        ]

        # 무시할 키워드
        ignore_keywords = [
            'icon', '아이콘', 'logo', '로고', 'header', '헤더', 'footer', '푸터',
            'button', '버튼', 'symbol', '심볼', 'decoration', '장식'
        ]

        # 중요 키워드가 있고 무시 키워드가 없으면 True
        has_important = any(keyword in context for keyword in important_keywords)
        has_ignore = any(keyword in context for keyword in ignore_keywords)

        return has_important and not has_ignore

    @staticmethod
    def is_important_image(img, markdown_content=None, image_position=None):
        """
        종합적인 필터링을 통해 이미지가 중요한지 판단

        여러 필터링 기준을 조합하여 이미지가 분석할 가치가 있는지 결정합니다.

        Args:
            img: PIL 이미지 객체
            markdown_content: 마크다운 텍스트 (선택적)
            image_position: 이미지 참조 위치 (선택적)

        Returns:
            bool: 중요한 이미지로 판단되면 True
        """
        # 1. 크기 기반 필터링
        if not ImageFilter.filter_by_size(img):
            logger.info("작은 크기로 필터링됨")
            return False

        # 2. 아이콘 필터링
        if ImageFilter.is_likely_icon(img):
            logger.info("아이콘으로 식별되어 필터링됨")
            return False

        # 3. 콘텐츠 복잡성 필터링
        if not ImageFilter.filter_by_content(img):
            logger.info("낮은 복잡성으로 필터링됨")
            return False

        # 4. 마크다운 컨텍스트 필터링 (정보가 있는 경우에만)
        if markdown_content and image_position is not None:
            if not ImageFilter.filter_by_context(markdown_content, image_position):
                logger.info("컨텍스트 기반으로 필터링됨")
                return False

        return True


class ChartAnalyzer:
    """
    차트 이미지를 분석하고 분석 결과를 마크다운에 삽입하는 클래스

    이 클래스는 문서(주로 PDF)를 마크다운으로 변환하고, 이미지를 추출하여 필터링한 후,
    다중 모달 LLM을 사용하여 차트나 다이어그램을 분석하고, 그 결과를 마크다운에 삽입하는
    전체 워크플로우를 처리합니다.
    """

    def __init__(self, llm_model_name: str = "gpt-4o-mini", image_resolution_scale: float = 2.0):
        """
        초기화 함수

        Args:
            llm_model_name: 사용할 LLM 모델 이름
            image_resolution_scale: 이미지 해상도 스케일 (1.0 = 72 DPI)
        """
        self.llm = ChatOpenAI(model=llm_model_name)
        self.image_resolution_scale = image_resolution_scale

    def convert_to_markdown(self, input_file_path: str, output_dir: str = "output") -> Tuple[str, DoclingDocument]:
        """
        문서를 마크다운으로 변환

        Args:
            input_file_path: 입력 파일 경로
            output_dir: 출력 디렉토리

        Returns:
            Tuple[str, DoclingDocument]: 마크다운 파일 경로와 Docling 문서 객체
        """
        # 출력 디렉토리 생성
        output_path = Path(output_dir)
        output_path.mkdir(parents=True, exist_ok=True)

        # PDF 파일 처리를 위한 옵션 설정
        pipeline_options = PdfPipelineOptions()
        pipeline_options.images_scale = self.image_resolution_scale
        pipeline_options.generate_page_images = True
        pipeline_options.generate_picture_images = True

        # DocumentConverter 설정
        doc_converter = DocumentConverter(
            format_options={
                InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
            }
        )

        # 문서 변환
        logger.info(f"문서 변환 시작: {input_file_path}")
        start_time = time.time()
        conv_res = doc_converter.convert(input_file_path)
        end_time = time.time()
        logger.info(f"문서 변환 완료: {end_time - start_time:.2f}초 소요")

        # 문서 파일명 추출
        input_path = Path(input_file_path)
        doc_filename = input_path.stem

        # 마크다운으로 저장 (외부 참조 이미지 사용)
        md_filename = output_path / f"{doc_filename}.md"
        conv_res.document.save_as_markdown(md_filename, image_mode=ImageRefMode.REFERENCED)

        return str(md_filename), conv_res.document

    def extract_images(self, document: DoclingDocument, output_dir: str) -> Dict[str, str]:
        """
        문서에서 이미지 추출 및 저장

        Args:
            document: Docling 문서 객체
            output_dir: 이미지를 저장할 디렉토리

        Returns:
            Dict[str, str]: 이미지 ID와 경로의 딕셔너리
        """
        output_path = Path(output_dir)
        output_path.mkdir(parents=True, exist_ok=True)

        image_paths = {}

        # 디버깅을 위한 로그
        logger.info(f"문서에서 이미지 추출 시작")

        # 표와 그림 추출
        table_counter = 0
        picture_counter = 0

        for element, _level in document.iterate_items():
            logger.info(f"요소 타입 발견: {type(element).__name__}")  # 디버그 로그

            if isinstance(element, TableItem):
                table_counter += 1
                image_id = f"table-{table_counter}"
                image_filename = output_path / f"{image_id}.png"

                try:
                    with image_filename.open("wb") as fp:
                        element.get_image(document).save(fp, "PNG")
                    image_paths[image_id] = str(image_filename)
                    logger.info(f"표 이미지 저장: {image_filename}")
                except Exception as e:
                    logger.error(f"표 이미지 저장 중 오류: {e}")

            if isinstance(element, PictureItem):
                picture_counter += 1
                image_id = f"picture-{picture_counter}"
                image_filename = output_path / f"{image_id}.png"

                try:
                    with image_filename.open("wb") as fp:
                        element.get_image(document).save(fp, "PNG")
                    image_paths[image_id] = str(image_filename)
                    logger.info(f"그림 이미지 저장: {image_filename}")
                except Exception as e:
                    logger.error(f"그림 이미지 저장 중 오류: {e}")

        logger.info(f"총 {len(image_paths)}개 이미지 추출 완료: {list(image_paths.keys())}")

        return image_paths

    def extract_images_with_filtering(self, document, output_dir, markdown_content=None):
        """
        문서에서 이미지 추출 및 저장 (종합적 필터링 적용)

        Args:
            document: Docling 문서 객체
            output_dir: 이미지를 저장할 디렉토리
            markdown_content: 마크다운 내용 (컨텍스트 필터링용)

        Returns:
            Dict[str, str]: 이미지 ID와 경로의 딕셔너리
        """
        output_path = Path(output_dir)
        output_path.mkdir(parents=True, exist_ok=True)

        image_paths = {}
        filtered_count = 0

        # 마크다운에서 이미지 참조 위치 추출 (있는 경우)
        image_positions = {}
        if markdown_content:
            for match in re.finditer(r'!\[(.*?)\]\((.*?)\)', markdown_content):
                pos = match.start()
                img_path = match.group(2)
                img_id = Path(img_path).stem
                image_positions[img_id] = pos

        # 표와 그림 추출
        for element, _level in document.iterate_items():
            if isinstance(element, (TableItem, PictureItem)) and hasattr(element, 'get_image'):
                try:
                    # 요소 ID 생성
                    element_type = "table" if isinstance(element, TableItem) else "picture"
                    element_id = getattr(element, 'id', str(id(element)))
                    image_id = f"image_{element_type}_{element_id}"

                    # 이미지 가져오기
                    img = element.get_image(document)

                    # 이미지 위치 찾기 (마크다운에서)
                    image_position = image_positions.get(image_id, None)

                    # 중요성 필터링
                    if not ImageFilter.is_important_image(img, markdown_content, image_position):
                        logger.info(f"필터링됨: {image_id}")
                        filtered_count += 1
                        continue

                    # 중요한 이미지만 저장
                    image_filename = output_path / f"{image_id}.png"
                    with image_filename.open("wb") as fp:
                        img.save(fp, "PNG")

                    image_paths[image_id] = str(image_filename)
                    logger.info(f"{element_type.capitalize()} 이미지 저장: {image_filename} ({img.size})")

                except Exception as e:
                    logger.error(f"이미지 처리 중 오류: {e}")

        logger.info(f"총 {len(image_paths)}개 이미지 추출 완료, {filtered_count}개 이미지 필터링됨")
        return image_paths

    def extract_image_references(self, markdown_content: str) -> List[Tuple[int, str, str]]:
        """
        마크다운 내용에서 이미지 참조를 추출

        마크다운에서 ![alt](path) 형식의 이미지 참조를 찾아 위치, ID, 경로를 추출합니다.

        Args:
            markdown_content: 마크다운 내용

        Returns:
            List[Tuple[int, str, str]]: (위치, 이미지 ID, 이미지 경로) 튜플 리스트
        """
        # 마크다운 이미지 참조 패턴: ![대체 텍스트](이미지 경로)
        pattern = r'!\[(.*?)\]\((.*?)\)'
        matches = re.finditer(pattern, markdown_content)

        image_references = []
        for match in matches:
            position = match.start()
            alt_text = match.group(1)
            image_path = match.group(2)

            logger.info(f"이미지 참조 발견: 대체 텍스트='{alt_text}', 경로='{image_path}'")

            # 이미지 ID 추출 (파일명에서 확장자 제외)
            image_id = Path(image_path).stem
            logger.info(f"추출된 이미지 ID: '{image_id}'")

            image_references.append((position, image_id, image_path))

        logger.info(f"마크다운에서 {len(image_references)}개의 이미지 참조를 찾았습니다.")
        return image_references

    def analyze_chart_with_multimodal_llm(self, image_path: str) -> str:
        """
        다중 모달 LLM을 사용하여 차트 이미지를 분석

        Args:
            image_path: 이미지 파일 경로

        Returns:
            str: 차트 분석 결과
        """
        try:
            # 이미지 파일 로드
            image = Image.open(image_path)

            # 이미지를 base64로 인코딩
            buffered = io.BytesIO()
            image.save(buffered, format="PNG")
            img_str = base64.b64encode(buffered.getvalue()).decode()

            # 다중 모달 메시지 구성
            message = HumanMessage(
                content=[
                    {
                        "type": "text",
                        "text": """
                        다음 차트 이미지를 자세하게 모두 한글로 5문장 이내로 설명해주세요.
                         - 차트 유형(막대, 선, 파이 등)을 먼저 설명해주세요.
                         - 핵심 데이터 트렌드 또는 패턴을 설명해주세요.
                         - 차트에서 볼 수 있는 주요 인사이트를 설명해주세요.
                         - 이미지가 일반 그림이나 다이어그램인 경우, 그림이 무엇을 나타내는지 설명해주세요.
                         - 차트 내의 내용만 나타내주고 의견이나 예측은 덧붙이지 말아주세요.
                         - 불렛포인트를 사용해서 출력해주세요.
                         # 출력 예시:
                            (차트 설명)
                            - 설명1
                            - 설명2
                            - 설명3
                            - 설명4
                            - 설명5
                        """
                    },
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/png;base64,{img_str}"},
                    },
                ],
            )

            # LLM 호출
            logger.info(f"차트 이미지 '{image_path}' 분석 시작")
            response = self.llm.invoke([message])
            logger.info(f"차트 이미지 '{image_path}' 분석 완료")

            return response.content

        except Exception as e:
            logger.error(f"이미지 분석 중 오류 발생: {e}")
            return f"이미지 분석 중 오류가 발생했습니다: {e}"

    def insert_analysis_into_markdown(self, markdown_content: str, image_references: List[Tuple[int, str, str]],
                                     analyses: Dict[str, str]) -> str:
        """
        차트 분석 결과를 원래 마크다운 내용의 적절한 위치에 삽입

        Args:
            markdown_content: 원본 마크다운 내용
            image_references: (위치, 이미지 ID, 이미지 경로) 튜플 리스트
            analyses: 이미지 ID를 키로 하고 분석 결과를 값으로 하는 딕셔너리

        Returns:
            str: 분석 결과가 삽입된 업데이트된 마크다운 내용
        """
        # 문자열 조작을 위해 리스트로 변환
        content_list = list(markdown_content)

        # 끝에서부터 삽입하여 이전 삽입으로 인한 위치 변화 방지
        offset = 0
        for position, image_id, image_path in sorted(image_references, reverse=True):
            if image_id in analyses:
                # 이미지 참조 다음에 분석 결과 삽입
                analysis_text = f"\n\n### 이미지 분석\n{analyses[image_id]}\n"

                # 이미지 참조 끝 위치 찾기
                end_pos = markdown_content.find(')', position) + 1

                # 분석 텍스트 삽입
                content_list[end_pos:end_pos] = list(analysis_text)
                offset += len(analysis_text)

        # 리스트를 문자열로 변환하여 반환
        return ''.join(content_list)

    def analyze_image_safe(self, image_path):
        """
        오류 처리가 강화된 이미지 분석 함수

        Args:
            image_path: 이미지 파일 경로

        Returns:
            str: 분석 결과 또는 오류 메시지
        """
        try:
            # 파일 존재 확인
            if not os.path.exists(image_path):
                return f"오류: 이미지 파일이 존재하지 않습니다 ({image_path})"

            # 파일 크기 확인
            file_size = os.path.getsize(image_path)
            if file_size == 0:
                return "오류: 이미지 파일이 비어있습니다"

            # 이미지 열기 시도
            try:
                img = Image.open(image_path)
                img.verify()  # 이미지 유효성 검사
            except Exception as e:
                return f"오류: 이미지 파일이 손상되었습니다 ({str(e)})"

            # 실제 분석 수행
            logger.info(f"LLM으로 이미지 분석 중: {image_path}")
            analysis = self.analyze_chart_with_multimodal_llm(image_path)
            logger.info("이미지 분석 완료")

            return analysis

        except Exception as e:
            error_msg = f"이미지 분석 중 예상치 못한 오류가 발생했습니다: {str(e)}"
            logger.error(error_msg)
            return error_msg

    def process_document(self, input_file_path: str, output_dir: str = "output") -> str:
        """
        전체 문서 처리 워크플로우

        1. 문서를 마크다운으로 변환
        2. 이미지 추출 및 필터링
        3. 마크다운에서 이미지 참조 추출
        4. 각 이미지 분석
        5. 분석 결과를 마크다운에 삽입

        Args:
            input_file_path: 입력 파일 경로
            output_dir: 출력 디렉토리

        Returns:
            str: 분석이 완료된 마크다운 파일 경로
        """
        print("\n===== 문서 처리 프로세스 시작 =====")

        # 출력 디렉토리 설정
        images_dir = os.path.join(output_dir, "images")
        print(f"이미지 저장 경로: {images_dir}")

        # 1. 문서를 마크다운으로 변환
        print("\n----- 1. 마크다운 변환 시작 -----")
        md_file_path, docling_document = self.convert_to_markdown(input_file_path, output_dir)
        print(f"마크다운 변환 완료: {md_file_path}")

        # 2. 마크다운 파일 로드
        print("\n----- 2. 마크다운 파일 로드 -----")
        with open(md_file_path, 'r', encoding='utf-8') as f:
            markdown_content = f.read()
        print(f"마크다운 내용 길이: {len(markdown_content)} 문자")

        # 3. 이미지 추출 및 필터링
        print("\n----- 3. 이미지 추출 및 필터링 -----")
        image_paths_by_id = self.extract_images_with_filtering(
            docling_document, images_dir, markdown_content
        )
        print(f"필터링 후 추출된 이미지 수: {len(image_paths_by_id)}")

        # 4. 이미지 참조 추출
        print("\n----- 4. 이미지 참조 추출 -----")
        image_references = self.extract_image_references(markdown_content)
        print(f"추출된 이미지 참조 수: {len(image_references)}")
        for i, (pos, img_id, img_path) in enumerate(image_references):
            print(f"참조 #{i+1}: 위치={pos}, ID={img_id}, 경로={img_path}")

        # 5. 각 이미지 분석 - 직접 경로 사용
        print("\n----- 5. 이미지 분석 시작 -----")
        analyses = {}
        for _, image_id, image_path in image_references:
            print(f"이미지 처리 시도: ID={image_id}, 경로={image_path}")

            # 마크다운에서 참조된 이미지 경로 확인
            full_image_path = image_path
            # 상대 경로인 경우 절대 경로로 변환
            if not os.path.isabs(image_path):
                # 마크다운 파일이 있는 디렉토리를 기준으로 경로 구성
                md_dir = os.path.dirname(md_file_path)
                full_image_path = os.path.join(md_dir, image_path)

            print(f"확인할 이미지 전체 경로: {full_image_path}")

            # 파일이 존재하는지 확인하고 분석
            if os.path.exists(full_image_path):
                print(f"이미지 파일 존재함, 분석 시작...")
                analysis = self.analyze_chart_with_multimodal_llm(full_image_path)
                analyses[image_id] = analysis
                print("분석 완료!")
            else:
                print(f"이미지 파일이 존재하지 않음: {full_image_path}")

        print(f"\n총 {len(analyses)}개 이미지 분석 완료")

        # 6. 분석 결과를 마크다운에 삽입
        print("\n----- 6. 분석 결과 마크다운에 삽입 -----")
        updated_markdown = self.insert_analysis_into_markdown(markdown_content, image_references, analyses)

        # 7. 결과 저장
        analyzed_md_path = md_file_path.replace('.md', '_analyzed.md')
        with open(analyzed_md_path, 'w', encoding='utf-8') as f:
            f.write(updated_markdown)

        print(f"\n===== 문서 처리 완료 =====")
        print(f"분석 결과가 '{analyzed_md_path}'에 저장되었습니다.")
        return analyzed_md_path

    def process_document_direct(self, input_file_path: str, output_dir: str = "output") -> str:
        """
        필터링과 분석을 직접 연결하는 문서 처리 함수

        이미지 ID 불일치 문제를 방지하기 위해 이미지 추출과 분석을 한 번에 처리합니다.

        Args:
            input_file_path: 입력 파일 경로
            output_dir: 출력 디렉토리

        Returns:
            str: 분석이 완료된 마크다운 파일 경로
        """
        # 출력 디렉토리 설정
        images_dir = os.path.join(output_dir, "images")
        Path(images_dir).mkdir(parents=True, exist_ok=True)

        print("\n===== 문서 처리 프로세스 시작 =====")

        # 1. 문서를 마크다운으로 변환
        print("1. 마크다운 변환 시작")
        md_file_path, docling_document = self.convert_to_markdown(input_file_path, output_dir)
        print(f"변환된 마크다운 파일: {md_file_path}")

        # 2. 마크다운 파일 로드
        print("2. 마크다운 파일 로드")
        with open(md_file_path, 'r', encoding='utf-8') as f:
            markdown_content = f.read()

        # 3. 마크다운에서 이미지 참조 추출
        print("3. 이미지 참조 추출")
        image_references = []
        for match in re.finditer(r'!\[(.*?)\]\((.*?)\)', markdown_content):
            position = match.start()
            alt_text = match.group(1)
            image_path = match.group(2)
            image_id = Path(image_path).stem
            image_references.append((position, image_id, image_path, alt_text))
            print(f"이미지 참조 발견: ID={image_id}, 경로={image_path}, 대체텍스트={alt_text}")

        # 4. 문서에서 이미지 추출 및 필터링과 동시에 분석 수행
        print("4. 이미지 추출, 필터링 및 분석")
        analyses = {}
        filtered_count = 0
        processed_count = 0

        # 모든 페이지, 표, 그림 요소 처리
        for element, _level in docling_document.iterate_items():
            # 이미지 요소 유형 확인
            if not isinstance(element, (TableItem, PictureItem)) or not hasattr(element, 'get_image'):
                continue

            try:
                # 요소 ID 및 유형 설정
                element_type = "table" if isinstance(element, TableItem) else "picture"
                element_id = getattr(element, 'id', str(id(element)))
                image_id = f"image_{element_type}_{element_id}"

                # 이미지 가져오기
                img = element.get_image(docling_document)

                # 이미지 크기 확인
                width, height = img.size
                print(f"이미지 추출: ID={image_id}, 크기={width}x{height}")

                # 이미지 필터링 적용
                if not ImageFilter.is_important_image(img):
                    print(f"이미지 필터링됨: {image_id}")
                    filtered_count += 1
                    continue

                # 필터링을 통과한 이미지 저장
                image_filename = os.path.join(images_dir, f"{image_id}.png")
                img.save(image_filename, "PNG")
                print(f"이미지 저장됨: {image_filename}")

                # 저장된 이미지 바로 분석
                print(f"이미지 분석 시작: {image_filename}")
                analysis = self.analyze_chart_with_multimodal_llm(image_filename)
                print("이미지 분석 완료!")

                # 이미지 ID와 분석 결과 저장
                analyses[image_id] = {
                    'analysis': analysis,
                    'position': None,  # 기본값 설정
                    'alt_text': element_type  # 기본 대체 텍스트
                }

                # 마크다운 참조와 매칭 시도 (옵션)
                for pos, ref_id, ref_path, alt_text in image_references:
                    if image_id in ref_id or ref_id in image_id:
                        analyses[image_id]['position'] = pos
                        analyses[image_id]['alt_text'] = alt_text
                        break

                processed_count += 1

            except Exception as e:
                print(f"이미지 처리 중 오류 발생: {e}")

        print(f"총 {processed_count}개 이미지 처리 완료, {filtered_count}개 이미지 필터링됨")

        # 5. 추가로 마크다운 참조 경로로 직접 이미지 확인
        print("5. 마크다운 참조 이미지 직접 분석")
        for pos, img_id, img_path, alt_text in image_references:
            # 이미 분석된 이미지는 건너뛰기
            if any(img_id in analyzed_id for analyzed_id in analyses.keys()):
                continue

            # 경로 확인
            full_image_path = img_path
            if not os.path.isabs(img_path):
                md_dir = os.path.dirname(md_file_path)
                full_image_path = os.path.join(md_dir, img_path)

            if os.path.exists(full_image_path):
                print(f"참조 이미지 발견: {full_image_path}")
                try:
                    # 이미지 로드하여 필터링
                    img = Image.open(full_image_path)
                    if not ImageFilter.is_important_image(img):
                        print(f"참조 이미지 필터링됨: {img_id}")
                        continue

                    # 분석 수행
                    analysis = self.analyze_chart_with_multimodal_llm(full_image_path)
                    analyses[img_id] = {
                        'analysis': analysis,
                        'position': pos,
                        'alt_text': alt_text
                    }
                    print(f"참조 이미지 분석 완료: {img_id}")
                except Exception as e:
                    print(f"참조 이미지 처리 중 오류: {e}")

        # 6. 분석 결과를 마크다운에 삽입
        print("6. 분석 결과 마크다운에 삽입")
        updated_markdown = markdown_content

        # 위치 정보가 있는 이미지는 해당 위치에 삽입
        positioned_analyses = {}
        for img_id, data in analyses.items():
            if data.get('position') is not None:
                positioned_analyses[img_id] = data['analysis']

        if positioned_analyses:
            image_refs = [(pos, id, path) for pos, id, path, _ in image_references]
            updated_markdown = self.insert_analysis_into_markdown(markdown_content, image_refs, positioned_analyses)

        # 위치 정보가 없는 이미지는 문서 끝에 추가
        unpositioned_items = [(img_id, data) for img_id, data in analyses.items()
                             if data.get('position') is None]

        if unpositioned_items:
            updated_markdown += "\n\n## 추가 이미지 분석\n"
            for img_id, data in unpositioned_items:
                updated_markdown += f"\n### 이미지 분석: {data.get('alt_text', 'Unknown')}\n{data['analysis']}\n"

        # 7. 결과 저장
        analyzed_md_path = md_file_path.replace('.md', '_analyzed.md')
        with open(analyzed_md_path, 'w', encoding='utf-8') as f:
            f.write(updated_markdown)

        print(f"분석이 완료되었으며 결과가 '{analyzed_md_path}'에 저장되었습니다.")
        return analyzed_md_path

    def process_document_with_loader(self, input_file_path: str, output_dir: str = "output") -> str:
        """
        DoclingLoader를 사용하여 전체 프로세스를 실행

        Args:
            input_file_path: 입력 파일 경로
            output_dir: 출력 디렉토리

        Returns:
            str: 분석이 완료된 마크다운 파일 경로
        """
        # 출력 디렉토리 설정
        Path(output_dir).mkdir(parents=True, exist_ok=True)
        images_dir = os.path.join(output_dir, "images")
        Path(images_dir).mkdir(parents=True, exist_ok=True)

        # 1. PDF 파일 처리를 위한 옵션 설정
        pipeline_options = PdfPipelineOptions()
        pipeline_options.images_scale = self.image_resolution_scale
        pipeline_options.generate_page_images = True
        pipeline_options.generate_picture_images = True

        # 2. DocumentConverter 설정
        doc_converter = DocumentConverter(
            format_options={
                InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
            }
        )

        # 3. DoclingLoader 설정
        docling_loader = DoclingLoader(
            file_path=input_file_path,
            converter=doc_converter,
            export_type=ExportType.MARKDOWN,
            md_export_kwargs={"image_mode": ImageRefMode.REFERENCED}
        )

        # 4. 문서 로드 및 마크다운 추출
        logger.info(f"DoclingLoader를 사용하여 문서 로드 시작: {input_file_path}")
        docs = list(docling_loader.lazy_load())

        if not docs:
            logger.error("문서 로드 실패: 결과가 없습니다.")
            return None

        markdown_content = docs[0].page_content

        # 5. 마크다운 파일 저장
        doc_filename = Path(input_file_path).stem
        md_file_path = os.path.join(output_dir, f"{doc_filename}.md")
        with open(md_file_path, 'w', encoding='utf-8') as f:
            f.write(markdown_content)

        # 6. 이미지 추출을 위해 원본 문서 변환 결과 가져오기
        conv_res = doc_converter.convert(input_file_path)

        # 7. 이미지 추출 및 저장
        image_paths_by_id = self.extract_images(conv_res.document, images_dir)

        # 8. 이미지 참조 추출
        image_references = self.extract_image_references(markdown_content)

        # 9. 각 이미지 분석
        analyses = {}
        for _, image_id, _ in image_references:
            if image_id in image_paths_by_id:
                image_path = image_paths_by_id[image_id]
                analysis = self.analyze_chart_with_multimodal_llm(image_path)
                analyses[image_id] = analysis

        # 10. 분석 결과를 마크다운에 삽입
        updated_markdown = self.insert_analysis_into_markdown(markdown_content, image_references, analyses)

        # 11. 결과 저장
        analyzed_md_path = md_file_path.replace('.md', '_analyzed.md')
        with open(analyzed_md_path, 'w', encoding='utf-8') as f:
            f.write(updated_markdown)

        logger.info(f"분석이 완료되었으며 결과가 '{analyzed_md_path}'에 저장되었습니다.")
        return analyzed_md_path

    def analyze_image_from_base64(self, base64_image: str) -> str:
        """
        Base64로 인코딩된 이미지를 직접 분석

        Args:
            base64_image: Base64로 인코딩된 이미지 문자열

        Returns:
            str: 분석 결과
        """
        try:
            # 다중 모달 메시지 구성
            message = HumanMessage(
                content=[
                    {
                        "type": "text",
                        "text": """
                        다음 차트 이미지를 자세하게 모두 한글로 5문장 이내로 설명해주세요.
                         - 차트 유형(막대, 선, 파이 등)을 먼저 설명해주세요.
                         - 핵심 데이터 트렌드 또는 패턴을 설명해주세요.
                         - 차트에서 볼 수 있는 주요 인사이트를 설명해주세요.
                         - 이미지가 일반 그림이나 다이어그램인 경우, 그림이 무엇을 나타내는지 설명해주세요.
                         - 차트 내의 내용만 나타내주고 의견이나 예측은 덧붙이지 말아주세요.
                         - 불렛포인트를 사용해서 출력해주세요.
                         # 출력 예시:
                            (차트 설명)
                            - 설명1
                            - 설명2
                            - 설명3
                            - 설명4
                            - 설명5
                        """
                    },
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                    },
                ],
            )

            # LLM 호출
            logger.info("Base64 이미지 분석 시작")
            response = self.llm.invoke([message])
            logger.info("Base64 이미지 분석 완료")

            return response.content

        except Exception as e:
            logger.error(f"Base64 이미지 분석 중 오류 발생: {e}")
            return f"이미지 분석 중 오류가 발생했습니다: {e}"

    def batch_process_with_metadata(self, input_file_path: str, output_dir: str = "output",
                              metadata_handler: Optional[callable] = None) -> Dict[str, Any]:
        """
        메타데이터 처리 기능을 포함한 배치 처리

        Args:
            input_file_path: 입력 파일 경로
            output_dir: 출력 디렉토리
            metadata_handler: 메타데이터 처리를 위한 콜백 함수

        Returns:
            Dict[str, Any]: 처리 결과 및 메타데이터
        """
        # 기본 처리
        result_path = self.process_document(input_file_path, output_dir)

        # 결과 정보
        result_info = {
            "input_file": input_file_path,
            "output_file": result_path,
            "processing_time": time.time(),
            "metadata": {}
        }

        # 메타데이터 처리 (제공된 경우)
        if metadata_handler and callable(metadata_handler):
            try:
                metadata = metadata_handler(input_file_path, result_path)
                result_info["metadata"] = metadata
            except Exception as e:
                logger.error(f"메타데이터 처리 중 오류: {e}")

        return result_info




In [60]:
# 사용 예시
def main(source_file_path):
    # 입력 파일 및 출력 디렉토리 설정
    input_file_path = source_file_path  # 분석할 PDF 파일 경로
    output_dir = "./output"  # 결과물이 저장될 디렉토리

    # ChartAnalyzer 인스턴스 생성 (다중 모달 모델 사용)
    analyzer = ChartAnalyzer(llm_model_name="gpt-4o-mini")

    # 방법 1: 기본 프로세스로 문서 처리
    result_path = analyzer.process_document(input_file_path, output_dir)
    print(f"분석된 파일 경로: {result_path}")

    # 방법 2: 직접 연결 방식 문서 처리 (이미지 ID 불일치 문제를 해결)
    # result_path = analyzer.process_document_direct(input_file_path, output_dir)
    # print(f"분석된 파일 경로: {result_path}")

    # 방법 3: Base64 인코딩된 이미지 직접 분석 예시
    # with open("chart_image.jpg", "rb") as img_file:
    #     img_data = base64.b64encode(img_file.read()).decode()
    #     analysis = analyzer.analyze_image_from_base64(img_data)
    #     print("분석 결과:", analysis)

    return result_path

In [61]:
main(source)


===== 문서 처리 프로세스 시작 =====
이미지 저장 경로: ./output/images

----- 1. 마크다운 변환 시작 -----
마크다운 변환 완료: output/2025 KB 부동산 보고서.md

----- 2. 이미지 추출 시작 -----
추출된 이미지 수: 164
추출된 이미지 ID 목록: ['picture-1', 'picture-2', 'picture-3', 'picture-4', 'picture-5', 'picture-6', 'picture-7', 'picture-8', 'picture-9', 'picture-10', 'picture-11', 'picture-12', 'picture-13', 'table-1', 'picture-14', 'picture-15', 'picture-16', 'picture-17', 'picture-18', 'picture-19', 'picture-20', 'picture-21', 'picture-22', 'table-2', 'picture-23', 'picture-24', 'picture-25', 'picture-26', 'table-3', 'table-4', 'picture-27', 'picture-28', 'picture-29', 'picture-30', 'picture-31', 'picture-32', 'picture-33', 'picture-34', 'picture-35', 'picture-36', 'picture-37', 'picture-38', 'table-5', 'picture-39', 'picture-40', 'picture-41', 'picture-42', 'picture-43', 'picture-44', 'picture-45', 'table-6', 'picture-46', 'picture-47', 'table-7', 'table-8', 'picture-48', 'picture-49', 'picture-50', 'picture-51', 'picture-52', 'picture-

In [65]:
analyzer = ChartAnalyzer(llm_model_name="gpt-4o-mini")
with open("/content/output/2025 KB 부동산 보고서_artifacts/image_000013_c460a7fb205cdb6a0616c776e13b7955b06a74530b77d06b146ebd074e7942e4.png", "rb") as img_file:
    img_data = base64.b64encode(img_file.read()).decode()
    analysis = analyzer.analyze_image_from_base64(img_data)
    print("분석 결과:", analysis)

분석 결과: - 이 차트는 선 그래프 형태로, 지역별 주택 매매가격 변동률을 나타냅니다.
- 두 개의 선은 각각 수도권(검정색)과 5개 광역시(노란색)의 주택 매매가격 변동률을 보여주며, 점선은 기타 지방의 변동률을 나타냅니다.
- 2006년부터 2024년까지의 데이터를 기반으로 하며, 주택 매매가격 변동률의 상승과 하락을 반복하는 패턴이 관찰됩니다.
- 특히, 2022년 중반부터 2023년 초까지 급격한 하락이 있었으며, 이후 2024년에는 다시 상승세를 보이고 있습니다.
- 전체적으로 수도권과 5개 광역시의 변동률이 비슷한 경향을 보이지만, 수도권이 상대적으로 높은 변동폭을 보이는 특징이 있습니다.
